In [1]:
!pip3 install apyori

  Running setup.py bdist_wheel for apyori ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Apriori Algorithm for Association Rule Mining

# ----------------------------------------------------------------
# ------------------ SETTING UP THE RANKED DATA ------------------
# ----------------------------------------------------------------

# Install assocation rule mining library
# pip3 install apyori

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

dataCsvPath = '/home/ec2-user/SageMaker/wsu-computer-science/CS-383_Cloud-Computing_2020-Spring/area-ranking/output/'
dailyRankedCrimes = pd.read_csv(dataCsvPath + "WorcesterData_BinRanked_Day.csv",error_bad_lines=False)
yearlyRankedCrimes = pd.read_csv(dataCsvPath + "WorcesterData_BinRanked_Year.csv",error_bad_lines=False)

# Fix the Column names in the daily ranked crimes
# Found that this breaks assocation rule mining - related to using date? 
#dailyRankedCrimes['ACTUAL_DTE'] = pd.to_datetime(dailyRankedCrimes['ACTUAL_DTE']*10000+dailyRankedCrimes['ACTUAL_DTE.1']*100+dailyRankedCrimes['ACTUAL_DTE.2'],format='%Y%m%d')
#del dailyRankedCrimes['ACTUAL_DTE.1']
#del dailyRankedCrimes['ACTUAL_DTE.2']
# Still an issue if converted to a nominal (category):
#dailyRankedCrimes['ACTUAL_DTE'] = dailyRankedCrimes['ACTUAL_DTE'].dt.strftime('%Y-%m-%d')
#dailyRankedCrimes['ACTUAL_DTE']          		= pd.Categorical(dailyRankedCrimes['ACTUAL_DTE'])

# Done Loading Data
print('You are ready to use the data')

You are ready to use the data


In [12]:
dailyRankedCrimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186124 entries, 0 to 186123
Data columns (total 7 columns):
ACTUAL_DTE                 186124 non-null int64
ACTUAL_DTE.1               186124 non-null int64
ACTUAL_DTE.2               186124 non-null int64
latbin                     186124 non-null float64
lonbin                     186124 non-null float64
WEIGHTED_INCIDENT_SCORE    186124 non-null int64
RANK                       186124 non-null float64
dtypes: float64(3), int64(4)
memory usage: 9.9 MB


In [9]:
# ------------------ DAILY DATASET --------------------

records = []
for i in range(0, 186123):
 records.append([str(dailyRankedCrimes.values[i,j]) for j in range(0, 4)])
 if(i%10000 == 0):
        print(i)

print("Created daily records")

# At this point it ran for many hours (on mny local machine) and never finished (not parallized) but on a i7 CPU with 16 GB of RAM
# We found that python arrays were just too inefficient 
#

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
Created daily records


In [10]:
association_rules = apriori(dailyRankedCrimes, min_support=0.2, min_confidence=0.95, min_lift=3, min_length=4)
association_results = list(association_rules)
print(len(association_results))

12


In [13]:
# ------------------ YEARLY DATASET --------------------

# Before deciding it was too inefficient I decided to try it with a smaller dataset - the Yearly Ranked Crimes Data

records2 = []
for i in range(0, 6961):
 records2.append([str(yearlyRankedCrimes.values[i,j]) for j in range(0, 5)])

association_rules = apriori(yearlyRankedCrimes, min_support=0.0045, min_confidence=0.95, min_lift=3, min_length=2)
association_results = list(association_rules)
print(len(association_results))
# 8741

# That is too many rules. I messed around with the variables and repeated
association_rules = apriori(yearlyRankedCrimes, min_support=0.2, min_confidence=0.95, min_lift=3, min_length=4)
association_results = list(association_rules)
print(len(association_results))
# 9

# I printed the Rules using the common commands (found in common-commands.py)

8741
9


In [14]:
# ----------------------------------------------------------------
# ---------- COMMON COMMANDS AFTER GENERATING THE RULES ----------
# ----------------------------------------------------------------

# View number of Rules
print(len(association_results))

# View first Rule
print(association_results[0])

# Print all rules
for item in association_results:
 # first index of the inner list
 # Contains base item and add item
 pair = item[0] 
 items = [x for x in pair]
 #print("Rule: " + items[0] + " -> " + items[1])
 print("Rule: " + str(list(item.ordered_statistics[0].items_base)) + " -> " + str(list(item.ordered_statistics[0].items_add)))
 #second index of the inner list
 print("Support: " + str(item[1]))
 #third index of the list located at 0th
 #of the third index of the inner list
 print("Confidence: " + str(item[2][0][2]))
 print("Lift: " + str(item[2][0][3]))
 print("=====================================")

9
RelationRecord(items=frozenset({'R', 'N'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({'N'}), items_add=frozenset({'R'}), confidence=1.0, lift=3.0), OrderedStatistic(items_base=frozenset({'R'}), items_add=frozenset({'N'}), confidence=1.0, lift=3.0)])
Rule: ['N'] -> ['R']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['b'] -> ['i']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['b'] -> ['l']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['i'] -> ['l']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['b'] -> ['i', 'l']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['b'] -> ['i', 'n']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['b'] -> ['l', 'n']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['i'] -> ['l', 'n']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.0
Rule: ['b'] -> ['i', 'l', 'n']
Support: 0.3333333333333333
Confidence: 1.0
Lift: 3.